# Lab 4 Statistical Interpretation of Uncertainty {-}

Name:
    
## Skills
1. Use Python to plot statistical distributions and histograms.
2. Use Python to calculate the mean, standard deviation, and standard deviation of the mean for a data set.
3. Explain the difference between standard deviation and standard deviation of the mean.
4. Use Python to read data from a file.

## Background Information

### Statistical Distributions
We've learned that all measurements have uncertainties associated with them.  This means that when a measurement (or experiment) is repeated, the obtained value is different every time.  Statisticians call this a random variable, but the numbers aren't as random as you might think.  You might think that the word random means that the value can be **anything**; any number at all from $-\infty \rightarrow \infty$ but that's not what we typically mean.  Instead, random in this context means a number that is random but subject to some probability rules.

The rules for deciding what random numbers are allowed are contained in a mathematical function called a probability distribution. The Gaussian (or Normal) distribution and Gamma distributions (pictured below) are two common distributions . (Students will no doubt be familiar with the term "bell curve".)  


In [ ]:
#| eval: true
#| echo: false
from scipy.stats import norm,gamma
from matplotlib import pyplot as plt
import numpy as np

x = np.arange(0,10,0.1)

fig,ax = plt.subplots(2,2)

μ = 5
σ = 1.3

ax[0][0].plot(x,norm.pdf(x,loc = μ,scale = σ),'r-',lw=4,alpha = 0.8)

a = 3

dataGamma = gamma.rvs(a,scale = 0.5,size = 10000)

ax[0][1].plot(x,gamma.pdf(x,a),'r-',lw=4,alpha = 0.8)



dataNorm = norm.rvs(loc = μ,scale = σ,size = 20)

ax[1][0].plot(dataNorm,[0.25 for x in dataNorm],'.',axes = None,ms = 8)
ax[1][0].set_xlim(2,7)
ax[1][0].set_aspect(8)
ax[1][0].spines['top'].set_visible(False)
ax[1][0].spines['left'].set_visible(False)
ax[1][0].spines['right'].set_visible(False)
ax[1][0].spines['bottom'].set_visible(False)
ax[1][0].set_yticks([])
ax[1][0].set_xticks([])
ax[0][0].set_title("Gaussian (Normal) Distribution")



dataGamma = gamma.rvs(a,size = 20)

ax[1][1].plot(dataGamma,[0.25 for x in dataGamma],'.',axes = None,ms = 8)
ax[1][1].set_xlim(2,7)
ax[1][1].set_aspect(8)
ax[1][1].spines['top'].set_visible(False)
ax[1][1].spines['left'].set_visible(False)
ax[1][1].spines['right'].set_visible(False)
ax[1][1].spines['bottom'].set_visible(False)
ax[1][1].set_yticks([])
ax[1][1].set_xticks([])
ax[0][1].set_title("Gamma Distribution")


plt.tight_layout()
plt.show()

The numbers on the horizontal axis are the possible random numbers and the height of the function indicates the probabilities. For example, for the Gaussian distribution numbers near 5 will be more likely to appear than numbers much greater or less than 5. Conversely, numbers greater than 8 and less than 2 will be quite unlikely to occur.  Numbers between 6 and 8 (or 2 and 4) are less likely than numbers between 4 and 6 but more likely than numbers greater than 8 (or less than 2).  You get the idea. The gamma distribution isn't symmetric like the Gaussian.  Numbers far above the peak location are more likely than numbers far below the peak location. 
The lower pane in the figure above is 20 random numbers selected (drawn) from this distribution. 


The mathematical expression for the Normal distribution is given by

$$
f(x) = {1\over \sigma \sqrt{2 \pi}} e^{-(x- \mu)^2 \over 2 \sigma^2}
$$

The shape and location of this function are controlled by two numbers: the mean ($\mu$) and standard deviation ($\sigma$).  The mean controls where the peak of the distribution is located and the standard deviation controls the width of the peak. One feature of the normal distribution is that 68% of the data lies between $\mu - \sigma$ and $\mu + \sigma$.  For this reason **we often assign the mean to be the expected value and the standard deviation to be the associated uncertainty**.

In [ ]:
#| eval: true
#| echo: false

from scipy.stats import norm
from matplotlib import pyplot as plt
import numpy as np

x = np.arange(0,10,0.1)

def makePlot(μ,σ,ax):
    data = norm.rvs(loc = μ,scale = σ,size = 10000)
    μ = np.mean(data)
    σ = np.std(data)
    height = norm.pdf(μ- σ,loc = μ,scale = σ)
    peak = norm.pdf(μ,loc = μ,scale = σ)
    ax.plot(x,norm.pdf(x,loc = μ,scale = σ),'r-',lw=4,alpha = 0.8)
    ax.hist(data,density = True,alpha = 0.2,bins = 20,edgecolor = "black")
    ax.annotate(text='', xy=(μ + σ,height), xytext=(μ - σ,height), arrowprops=dict(arrowstyle='<->'))
    ax.annotate(text='68%', xy=(μ - 0.4 * σ,0.4*height), xytext=(μ - 0.4 * σ,0.4*height),fontsize = 16)
    ax.annotate(text=f"μ = {μ:3.1f}",xy=(8,0.75 * peak),xytext=(8,0.75 * peak),fontsize = 15)
    ax.annotate(text=f"σ = {σ:3.1f}",xy=(8,0.5 * peak),xytext=(8,0.5 * peak),fontsize = 15)
    ax.set_xlim(0,10)
    
fig,ax = plt.subplots(4)

makePlot(5,1.3,ax[0])
makePlot(5.5,0.9,ax[1])
makePlot(7.5,0.4,ax[2])
makePlot(3.0,1.5,ax[3])
plt.tight_layout()
plt.show()

So what does this have to do with uncertainties in scientific experiments? When a scientist performs measurements and gathers data, **they are gathering samples from a statistical distribution, usually a Gaussian.** This idea is crucial to understanding how statistical distributions describe uncertainty.  The challenge is figuring out which Gaussian distribution the data came from.  


### Histograms
When an experiment can be repeated over and over again under identical physical conditions, the data can be visualized using something called a histogram.  To construct a histogram, the range of plausible measurement values is first divided into bins.  Each measurement is then assigned a bin and the total number of measurements in each bin is updated.  The number of measurements in all of the bins can be visualized by plotting a bar for each bin, with the height of the bar indicating how many measurements were in the bin.  Below is a histogram for 10 repetitions of an experiment to determine the acceleration due to gravity.

In [ ]:
#| echo: false
#| eval: true
#| output: true
import matplotlib.pyplot as plt
import numpy.random as rand

data = rand.normal(10.5,1.25,10) # Dummy data just for this example.

ax = plt.axes()
ax.hist(data,bins = 20,color = 'blue',edgecolor = 'black')
ax.set_xlabel("g (m/s^2)")
plt.show()

Ten data points isn't a lot but you can probably start to form a reasonable guess for the mean(expected value) and standard deviation(uncertainty) of the underlying distribution.  As more data is gathered, your estimates for $\mu$ and $\sigma$ will update and your confidence in those values increases with it.  (Illustrated in the figure below)


In [ ]:
#| eval: true
#| echo: false
#| fig-cap: "As more measurements are performed the level of confidence in the mean and standard deviation increase"

from scipy.stats import norm
import matplotlib.pyplot as plt
import numpy.random as rand
import numpy as np

x = np.arange(2,15,0.1)

fig,ax = plt.subplots(4,1,figsize= (4,12))

#ax = plt.axes()
ax[0].hist(rand.normal(10.5,1.25,10),bins = 20,color = 'blue',edgecolor = 'black',density = True)
ax[0].plot(x,norm.pdf(x,loc = 10,scale = 2),'r-',lw=4,alpha = 0.8)
ax[0].set_xlabel("g (m/s^2)")
ax[0].set_title("10 samples")
ax[0].annotate(text=f"μ = {10:3.1f}",xy=(2.5,0.7),xytext=(2.5,0.7),fontsize = 15)
ax[0].annotate(text=f"σ = {2:3.1f}",xy=(2.5,0.5),xytext=(2.5,0.5),fontsize = 15)
#ax[0].set_aspect(0.8)

ax[1].hist(rand.normal(10.5,1.25,100),bins = 20,color = 'blue',edgecolor = 'black',density = True)
ax[1].plot(x,norm.pdf(x,loc = 10.15,scale = 1),'r-',lw=4,alpha = 0.8)
ax[1].set_xlabel("g (m/s^2)")
ax[1].set_title("100 samples")
ax[1].annotate(text=f"μ = {10.15:3.1f}",xy=(2.5,0.3),xytext=(2.5,0.3),fontsize = 15)
ax[1].annotate(text=f"σ = {1:3.1f}",xy=(2.5,0.2),xytext=(2.5,0.2),fontsize = 15)
#ax[1].set_aspect(0.3)

ax[2].hist(rand.normal(10.5,1.25,1000),bins = 20,color = 'blue',edgecolor = 'black',density = True)
ax[2].plot(x,norm.pdf(x,loc = 10.3,scale = 1.12),'r-',lw=4,alpha = 0.8)
ax[2].set_xlabel("g (m/s^2)")
ax[2].set_title("1000 samples")
ax[2].annotate(text=f"μ = {10.3:3.1f}",xy=(2.5,0.3),xytext=(2.5,0.3),fontsize = 15)
ax[2].annotate(text=f"σ = {1.12:3.2f}",xy=(2.5,0.2),xytext=(2.5,0.2),fontsize = 15)
#ax[2].set_aspect(0.045)

ax[3].hist(rand.normal(10.5,1.25,10000),bins = 20,color = 'blue',edgecolor = 'black',density = True)
ax[3].plot(x,norm.pdf(x,loc = 10.5,scale = 1.25),'r-',lw=4,alpha = 0.8)
ax[3].set_xlabel("g (m/s^2)")
ax[3].set_title("10000 samples")
ax[3].annotate(text=f"μ = {10.5:3.1f}",xy=(2.5,0.3),xytext=(2.5,0.3),fontsize = 15)
ax[3].annotate(text=f"σ = {1.25:3.2f}",xy=(2.5,0.2),xytext=(2.5,0.2),fontsize = 15)
fig.tight_layout()
plt.show()

The location of the distribution's peak would be the expected value and the spread of the peak (at about half the max height) would be the associated uncertainty.


### Mean
The peak of the distribution is located at the mean ($\mu$) and can be calculated from a data set using the following expression

$$
\begin{align*}
\mu &= {x_1 + x_2 + \dots\over N}\\
&= {1\over N}\sum_{i=1}^N \\
\end{align*}
$$


The mean should be used as the expected value when reporting results.  The uncertainty in the mean will decrease as more measurements are added to the data set.


### Standard Deviation
The standard deviation indicates the spread in the distribution and can be calculated using the following expression
$$
\sigma = \sqrt{ \sum_{i=1}^N (x_i - \mu)^2 \over N}
$$


The standard deviation gives the uncertainty in **a single measurement**.  If the experiment were repeated again, under identical conditions, the standard deviation would be the uncertainty associated with any single measurement.  Just as with the mean, the standard deviation becomes more reliable (smaller uncertainty) as the size of the data set increases.

Where did this expression for the standard deviation come from?  A full derivation won't be shown here (see me if you'd like more information), but some brief motivation for why this expression is a measure of the spread in the data will be given.  First notice the quantity $x_i - \mu$, which is a measure of how far away each data point is from the mean.  Summing up these differences would be some indication to the amount of spread in the data. However, since some of these differences will be positive and some negative, summing them could result in zero even when the spread in the data is not zero.  Instead the square of the differences is summed: $\sum_{i=1}^N (x_i - \mu)^2$ and then divided by $N$ to get an average.  
$$
{1\over N} \sum_{i=1}^N (x_i - \mu)^2
$$

The final expression is the square root of this expression: 

$$
\sigma = \sqrt{{1\over N} \sum_{i=1}^N (x_i - \mu)^2}
$$  

### Standard Deviation of the Mean
Notice that the standard deviation was not assigned to be the uncertainty in the mean.  The reasoning for this is subtle. The shape of the histogram improves with each data point added and therefore the uncertainty associated with the mean should decrease also.  However, the standard deviation does not decrease as $N$ increases.  Even though $N$ is on the denominator of the expression, as $N$ increases so too does the numerator. Hence, both the numerator and denominator increase together.  

The uncertainty in the mean is given by  

$$
\sigma_N = {\sigma \over \sqrt{N}}
$$

which decreases as more data points are added. This is the quantity that we should assign to be the uncertainty of the mean.

### Comparing Results
In lab #1, the success of an experiment was decided by comparing the fractional uncertainty to the percent error. If the fractional uncertainty was bigger than the percent error then the experiment was judged to be a success.  While this criteria isn't wrong, it doesn't tell us how good or bad the agreement is and it also doesn't provide any boundary between acceptable and unacceptable results. We are now in a position to describe a more robust criteria for judging success.

Recall that every time an experiment is performed the measurements gathered are random draws from a distribution.  This distribution is assumed to be **centered at the expected value ($x_e$)**. If the results of the experiment (usually the mean) are too far away from the expected mean, the probability that the measurements were random draws from the assumed distribution is so small that we lose confidence in that assumption.  The figure below illustrates this idea.  The red curve is the assumed distribution centered at the expected value and the blue bars indicate the measurements during the experiment.  The mean of these results is indicated by the red dashed curve and the expected mean is indicated by the green dashed curve.

In [ ]:
#| eval: true
#| echo: false

from matplotlib import pyplot as plt
from numpy import arange,exp,linspace,random,mean

x = arange(1420,1460,.1)
y = 10*exp(-(x-1448)**2/(2 * 5**2))

data = random.normal(1440,5,20)
μ = mean(data)
plt.hist(data,bins = 15,alpha = 0.7,rwidth = 0.85,color='#0504aa')
plt.plot(x,y,'r-')
plt.plot([μ for x in range(5)],linspace(0,15,5),'r--') 
plt.plot([1448 for x in range(5)],linspace(0,15,5),'g--') 
plt.xlabel("Speed of Sound")
plt.ylabel("No. of occurrences")
plt.ylim(0,14)
plt.annotate(text=fr"$x_e = 1448$",xy=(1445,11),xytext=(1445,11),fontsize = 15)
plt.annotate(text=fr"$x_b = $ {μ:4.0f}",xy=(μ-3,11),xytext=(μ-3,13),fontsize = 15)
plt.annotate(text='', xy=(1448,9), xytext=(1448,11), arrowprops=dict(arrowstyle='->'))
plt.annotate(text='', xy=(μ,11), xytext=(μ,13), arrowprops=dict(arrowstyle='->'))

plt.show()

To determine the probability that these data points came from the assumed distribution we first calculate the quantity

$$ t = {|x_b - x_e|\over \sigma_n}$$

which measures how many standard deviations of the mean the measured result differs from the expected result.  Here $x_b$ is the best result from your experiment(usually the mean of your dataset), $x_e$ is the expected value, and $\sigma_n$ is the standard deviation of the mean.

 For values of $t <1$ the experiment is probably a success and for $t \gg 1$ the experiment probably isn't a success.  But what about intermediate values?  It would be nice to make a more probabilistic statement about the success of our experiment.  Something like, "The probability that the results of our experiment were drawn from a distribution with the expected result as the mean is 90\%"

We can use Python to answer this question using the code given below.  

In [ ]:
from scipy.stats import ttest_1samp
results = [1.64,1.57,1.72,1.55,1.68,1.62,1.70,1.59]

expected_value = 1.67
P = ttest_1samp(results,expected_value)

print(P.pvalue)

The result is called a p-value and in this case it is equal to $0.144$ or $14.4\%$.  This means that the probability that $x_b$ was a random draw from the assumed distribution(centered at $x_e$) is $14.4\%$

The boundary between acceptable and unacceptable depends on the level below which we judge a discrepancy to be unreasonably probable.   This really is a matter of opinion but often a community of people will agree on a certain level.  Many scientists regard $5\%$ to be the appropriate boundary and if the p-value is smaller than $5\%$ the discrepancy is deemed significant. In other words, the probability of obtaining the result as a random draw from a distribution centered at the expected value is so small that it probably didn't.  In this case, the scientist ought to look for mistakes in the experimental setup.

On the other hand, if the p-value is above the boundary, the discrepancy is deemed insignificant and the experiment is deemed a success.  Note that p-values do not confirm the hypothesis, but just fail to reject it.

A p-value for two data sets can be calculated to see if their distributions are statistically the same.  As before the relevant quantity is 

$$ t = {|x_b - x_e|\over \sigma_\text{joint}}$$

but the standard deviation is now given by

$$\sigma_\text{joint} = \sqrt{{N_1 + N_2\over N_1 N_2} \cdot {(N_1 - 1)\sigma_1^2 + (N_2 - 1)\sigma_2^2 \over N_1 + N_2-2}}$$

Fortunately, python can do all of these calculations for us.  For example, if two groups of students measure the time it takes for a ball to fall $3.00$ m, you could use the code below to calculate a p-value.

In [ ]:
from scipy.stats import ttest_ind
dataOne = [0.75,0.74,0.79,0.76,0.77,0.73,0.74,0.78,0.76,0.75]
dataTwo = [0.81,0.80,0.79,0.76,0.77,0.80]

P = ttest_ind(dataOne,dataTwo)

print(f"The p-value for these two data sets is {P.pvalue:8.4f} = {P.pvalue*100:4.2f} %.")

Since the p-value is less than the $5\%$ boundary we conclude that the results from these two experiments differ significantly.

## Useful Plots

### Histograms
To plot a histogram in Python you should use the `pyplot.hist(data)` function and pass your data set into the function.  There are several optional arguments to the `hist` function that can help customize the appearance. A few of the more important ones are given in the table below

|Optional Argument| Description| Example|
|-----------------|------------|--------|
| bins | Specify how many bins should be displayed.| bins = 20|
| color| Specify what color the bars should be.| color = "blue"|
|edgecolor| Specify the color of the edge of the bars.| edgecolor = "red"|



Some example code for building a histogram is given below.


In [ ]:
import matplotlib.pyplot as plt
import numpy.random as rand

data = rand.normal(5,.5,1000) # Dummy data just for this example.

ax = plt.axes()
ax.hist(data,bins = 20,color = 'blue',edgecolor = 'black')
plt.show()

### Plotting functions
Plotting a function in Python can be accomplished by following these steps:
1. Build a list of x value (horizontal axis).
2. Calculate the function value for every x value.
3. Use `pyplot.plot(x,y)` to plot the x-y pairs.
4. Use `pyplot.show()` to display the plot.

Several options to the `pyplot.plot()` function will allow you to customize the plot.  Some common ones are listed below:
1. `linestyle`: specify what the line representing the function looks like.
2. `color`: Specify the color of the line representing the function.
3. `linewidth`: Specify the thickness of the line representing the function.

The functions `ax.set_xlabel()`, `ax.set_ylabel()`, and `ax.set_title()` can also be used to specify axes labels and a plot title.  An example is given in the code cell below.

In [ ]:
#| eval: true
#| echo: true
import matplotlib.pyplot as plt
import numpy as np

x = np.arange(0,2 * np.pi,0.1) # Step 1
y = np.sin(x)                  # Step 2
ax = plt.axes()
ax.plot(x,y,linewidth = 4,color = 'black',linestyle = "--")  # Step 3
ax.set_xlabel("x")
ax.set_ylabel("f(x)")
ax.set_title("f vs x")
plt.show()                     # Step 4

## Activity I Pendulum Period (33 points)

### Equpiment needed
1. 1.5-meter-long pendulum.
2. Stopwatch (the one on your phone will do)

### Goal (Overview)
Each student will measure the period of a 1.5-meter-long pendulum 10 times. This will result in a set of ~150 data points for the whole class.  We will then calculate the acceleration due to gravity for each measurement and calculate the  i) mean, ii) standard deviation, and iii) standard deviation of the mean and use these results to report an appropriate uncertainty.

### Procedure
1. A pendulum that is 1.5 meters long will be set up at the front of the class. Each person should measure the period of the pendulum with a stopwatch 10 times. Data should be recorded to the nearest 0.01 s.  For the best results try to keep the pendulum's amplitude consistent for all of the measurements, even by restarting as necessary.  The amplitude should be relatively small (less than about $10^\circ$).  The pendulum should be swinging (rather than at rest) when the time measurement begins.
2. Record your 10 measurements in the google sheets link provided below.
3. Once everyone has collected their measurements someone needs to move all of the data into a single column.  Then download the file to your computer and use the `np.loadtxt()` function to read this data into Python. Do this on line 7 of the code cell below.  (Remember: The file must be located in the same directory as this Jupyter notebook.)
4. The acceleration due to gravity can be calculated from $$ g = {4 \pi^2 L \over T^2}$$ On line 9 of the code cell below calculate the acceleration due to gravity for each period measurement in the data set. To include $\pi$ in your calculation use `np.pi`.
5. Use equations 4.5, 4.9, and 4.14 from the book to calculate the mean, standard deviation, and standard deviation of the mean.  Perform these calculations on lines 10, 11, and 12 of the code cell below.  Use `np.sum()` to perform summations and `np.sqrt()` to perform square root functions.
6. On lines 14,15, and 16 in the code cell below use the functions `stats.mean()` and `stats.stdev()` to calculate the mean, standard deviation, and standard deviation of the mean.  
7. Add a formatted print statement (see examples from previous labs) to report an appropriate value for $g$ with its uncertainty.
8. Plot a histogram of the g values.  
9. The normal distribution is given by $$ N(x) = {1\over \sigma \sqrt{2 \pi}} e^{-(x- \mu)^2\over 2 \sigma^2}$$. Plot this distribution on top of the histogram. Verify that the limiting distribution matches the histogram.  
10. Calculate the p-value for this dataset using $g= 9.8$ as the expected value.  State whether the deviation of your experimental results are significant enough (at the $5\%$ level ) to reject the experiment. Do this on line 17 in the code cell below.

[Pendulum Data](https://docs.google.com/spreadsheets/d/1A2Xb7dDaDxjSH8YzRn0-Lb2er6qxWySYjmuLK-yiP4w/edit?usp=sharing)

In [ ]:
#| eval: false
#| echo: true

import numpy as np
import statistics as stats
import math as mt

%matplotlib inline

T = np.loadtxt("pendulum_data.txt",float)  # Line 7
N = len(T)     # Find the number of data points.
g =            # Line 9
meanG =        # Line 10
stdG =         # Line 11
stdMeanG =     # Line 12

μ =            # Line 14 
σ =            # Line 15
σ_μ =          # Line 16
p_value =      # Line 17



ax = plt.axes()
# Plot the histogram and Normal distribution below.

plt.plot(x,norm_data)
plt.xlabel('g (m/s^2)')
plt.ylabel('Probability')
plt.show()

## Activity II - Radioactivity (33 points)

### Equipment needed
1. Geiger Counter
2. Radioactive sample (don't worry, they aren't active enough to be dangerous)
3. LabQuest mini box with USB cable.
4. Logger Pro software.  You can download a copy for your personal machine ([windows](https://www.vernier.com/d/yxoaa) or [mac](https://www.vernier.com/d/u7caf) ) or use one of the lab computers which have Logger Pro installed already.

### Goal (Overview)
We will use the electronic radiation monitor to measure the number of decays for a radiocative sample. We will calculate the mean and standard deviation of the data and make a histogram of the data.  The Poisson distribution will be plotted on top of the histogram to show it as the limiting distribution.

### Procedure
1. Using the Vernier Radiation Monitor, measure the decays of a radioactive sample.  Below you will find the instructions for setting up the Vernier Radiation Monitor to your laptop.  Use 10-second intervals and make 100 measurements.  
2. Copy the data from Logger Pro to an Excel spreadsheet and save the worksheet.(The worksheet should have only one column of data.  Don't include the time data in the Excel worksheet.)
3. Use the equation above for the mean ($\mu$) to calculate the mean of the data and the equation below to calculate the standard deviation of the data. $$\sigma = \sqrt{\mu}$$. Perform these calculations on lines 7 and 8 in the code cell below.
4. Add a formatted print statement to report the appropriate count number with its associated uncertainty.
5. One line 11 of the code cell below, produce a histogram just as we did together in activity I. Choose `bins = ` so that the histogram displays a sufficient amount of detail.
6. The limiting distribution for this kind of data is called a Poisson distribution (equation given below).  It is only defined for interger values of the argument and it is hard to plot using standard plotting techniques.  I have provided the code to plot this function in the code cell below.  You may have to modify the numbers on line 14 to plot over the appropriate range.  $$P(x) = e^{-\mu} {\mu^x \over x!} $$

>Instructions for using the Vernier Radiation Monitor:

>1. Connect the Vernier Radiation Monitor to the DIG 1 port of the LabQuest Mini.
>2. Connect the USB cord from the LabQuest Mini to your laptop.
>3. Within the LoggerPro software, do the following:
>   1. Select Experiment -> Data Collection and change the duration to 1000 seconds.
   
   
   
   <img src="https://github.com/lancejnelson/PH121/raw/gh-pages/files/LoggerProRadiation.png" alt="drawing" width="600px"/>

In [ ]:
#| eval: false
#| echo: true

from scipy.stats import poisson,norm
from numpy import arange
from matplotlib import pyplot as plt

#data = np.loadtxt("radiation_data.txt",float)

#μ =   # Line 7
#σ =   # Line 8

ax = plt.axes()
              # Line 11

dist = poisson(μ)
x = arange(0,40)    # Line 14: May have to modify these numbers.

ax.plot(x,dist.pmf(x))
plt.show()


## Activity III - Projectile Range (33 points)

### Goal (Overview)
This activity is identical in nature to Activity I.  Instead of measuring the period of a pendulum, we will measure the range of a projectile.  The analysis of the data will be very similar to that performed in Activity I. 

### Procedure
1. Use kinematics to predict the range of the steel ball.  Perform the calculations in the code cell below. 
2. Using the projectile launcher, fire a steel ball 40 times (10 launches per student) and measure the range in meters.  The projectile should be placed on the table.  Use an initial launch angle of $30^\circ$ and use the third spring setting to produce the largest range.
3. Again, we will pool our data together as a class to increase the size of the dataset.  Therefore, it is very important that every student launches **under the same conditions**.  Add your data to the Google sheets document linked below.
4. When all the data is collected, have Python read the data set using `np.loadtxt()`.
5. Calculate the mean, standard deviation, and standard deviation of the mean using your preferred calculation method. (Either with `stats.mean()` and `stats.stdev()` or brute force calcuation of the equations in the book.)
6. Plot a histogram of the data.  Choose `bins =` appropriately so the histogram displays with sufficient detail.
7. Plot the normal distribution on top to verify that it is the limiting distribution.
8. Add a formatted print statement to report the particle's range with its associated uncertainty.
9. Calculate the p-value for your data set using the predicted range from part 1 as your expected value. State whether the deviation of your experimental results are significant enough (at the $5\%$ level ) to reject the experiment. 

No template code cell will be provided for this activity (hopefully you are feeling more and more comfortable writing simple code on your own), but you can copy/paste from previous activities.

[Projectile Data](https://docs.google.com/spreadsheets/d/1frC8Yy7TK7fDN_z7E-1BQEOr6knhIZweFHdOTky6TQM/edit?usp=sharing)


In [ ]:
# Put code here
